In [1]:
import syft as sy
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
hook = sy.TorchHook(torch)

W0820 20:37:22.182676 140492295587136 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0820 20:37:22.195190 140492295587136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
from grid.workers.socketio_client import WebsocketIOClientWorker

# Config serialiser and compression strategies
sy.torch_serde._serialize_tensor = sy.torch_serde.numpy_tensor_serializer
sy.torch_serde._deserialize_tensor = sy.torch_serde.numpy_tensor_deserializer
sy.serde.serde._apply_compress_scheme = sy.serde.apply_no_compression


In [3]:
websocket_workers = {}
for i in [1,2,3,4]:
    print("Initilalize worker #"+str(i))
    worker = WebsocketIOClientWorker(hook, host='localhost', port=int(5000+i), id="worker"+str(i))
    websocket_workers[i] = worker

Initilalize worker #1
Initilalize worker #2
Initilalize worker #3
Initilalize worker #4


In [4]:
for k,v in websocket_workers.items():
    print("Connect worker #"+str(k))
    websocket_workers[k].connect()
    
    

Connect worker #1
Connect worker #2
Connect worker #3
Connect worker #4


In [7]:
x = torch.tensor([45., 42, 73, 34, 51, 454, 656, 56])
y = torch.tensor([10,20,30,40,50.,11,12,13,14,15])
z = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [7, 8, 9]], dtype=torch.float)
x_ptrs={}
y_ptrs={}
z_ptrs={}
for k,v in websocket_workers.items():
    print("Send tensors to worker #"+str(k))
    x_ptrs[k] = x.send(v)
    y_ptrs[k] = y.send(v)
    z_ptrs[k] = z.send(v)

Send tensors to worker #1
Send tensors to worker #2
Send tensors to worker #3
Send tensors to worker #4


In [8]:
for k,v in x_ptrs.items():
    print(v)
for k,v in y_ptrs.items():
    print(v)
for k,v in z_ptrs.items():
    print(v)

(Wrapper)>[PointerTensor | me:31829686313 -> worker1:18818291869]
(Wrapper)>[PointerTensor | me:64710365573 -> worker2:18818291869]
(Wrapper)>[PointerTensor | me:14995117796 -> worker3:18818291869]
(Wrapper)>[PointerTensor | me:62467162180 -> worker4:18818291869]
(Wrapper)>[PointerTensor | me:21468031359 -> worker1:64164938710]
(Wrapper)>[PointerTensor | me:53368236691 -> worker2:64164938710]
(Wrapper)>[PointerTensor | me:5282950147 -> worker3:64164938710]
(Wrapper)>[PointerTensor | me:30764936046 -> worker4:64164938710]
(Wrapper)>[PointerTensor | me:26038261408 -> worker1:94820936017]
(Wrapper)>[PointerTensor | me:72182574044 -> worker2:94820936017]
(Wrapper)>[PointerTensor | me:17342396424 -> worker3:94820936017]
(Wrapper)>[PointerTensor | me:19040667037 -> worker4:94820936017]


In [ ]:
#z_ptr = x_ptr + y_ptr

In [ ]:
#z = z_ptr.get()

In [ ]:
z